In [1]:
import matplotlib
from pianoq_results import ScanResult
from simulations.mplc_sim import MPLCSimResult

matplotlib.use('TKAgg')
%load_ext autoreload
%autoreload 2
from lab.mplc.discrete_scan_result import DiscreetScanResult
from misc.misc import run_in_thread_simple, run_in_thread
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_modes2 import gen_input_spots_array, gen_output_modes_Unitary
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.mask_utils import remove_input_modes
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_QKD_masks, create_WFM_unitary_masks
from pianoq.misc.misc import detect_gaussian_spots_subpixel
import time 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

d_no = 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate masks WFM

In [2]:
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_diffuser_masks

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path =    fr'{dir_path}\{timestamp}_diffuser_d{d_no}d.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_diffuser_d{d_no}d.mplcsim'
mplc_sim = create_WFM_diffuser_masks(N_iterations=15, out_path=masks_path)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

100%|██████████| 15/15 [04:11<00:00, 16.78s/it]


In [28]:
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCSimResult
path = glob.glob(rf'{dir_path}\*diffuser_d{d_no}d.mplcsim')[0]
mr = MPLCSimResult()
mr.loadfrom(path)
mimshow((np.abs((mr.forward_fields[-1, :]).sum(axis=0))**2)[mr.active_slice])

(<Figure size 640x480 with 2 Axes>, <Axes: >)

# HW

In [2]:
# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

Got MPLC


In [2]:
# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

# Motors
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=10, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)
print('Got Time tagger')

Got MPLC
Got Thorlabs motors
Got Zaber motors
Got Time tagger


# Single counts large scan

In [ ]:
masks_path = glob.glob(rf'{dir_path}\*diffuser_d{d_no}d*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
mplc.load_masks(masks, linear_tilts=True)

integration_time = 1
tt.set_integration_time(integration_time)

# scan signal
start_x = 10.2
end_x = 13.2
start_y = 7.7
end_y = 11.2

x_pixels = 30
y_pixels = 35
pixel_size_x = 0.1
pixel_size_y = 0.1

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = f'{dir_path}\\{timestamp}_singles_signal_d{d_no}d_exp_{integration_time}s.scan'
scanner_sig = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, 
                        is_timetagger=True, coin_window=2e-9, saveto_path=path)

scanner_sig.scan(x_motor=mxs, y_motor=mys, ph=tt)


# scan idler
start_x = 7.75
end_x = 10.75
start_y = 0.5
end_y = 4

x_pixels = 30
y_pixels = 35
pixel_size_x = 0.1
pixel_size_y = 0.1


timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = f'{dir_path}\\{timestamp}_singles_idler_d{d_no}d_exp_{integration_time}s.scan'
scanner_idl = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, 
                        is_timetagger=True, coin_window=2e-9, saveto_path=path)

scanner_idl.scan(x_motor=mxi, y_motor=myi, ph=tt)


Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 368. Singles2: 512. Coincidence: 0.00.
dur: 1. pix: 0, 1. Singles1: 389. Singles2: 569. Coincidence: 0.00.
dur: 3. pix: 0, 2. Singles1: 344. Singles2: 635. Coincidence: 1.00.
dur: 5. pix: 0, 3. Singles1: 342. Singles2: 728. Coincidence: 0.00.
dur: 6. pix: 0, 4. Singles1: 368. Singles2: 774. Coincidence: 0.00.
dur: 8. pix: 0, 5. Singles1: 369. Singles2: 853. Coincidence: 0.00.
dur: 9. pix: 0, 6. Singles1: 350. Singles2: 1211. Coincidence: 0.00.
dur: 11. pix: 0, 7. Singles1: 343. Singles2: 1462. Coincidence: 0.00.
dur: 12. pix: 0, 8. Singles1: 322. Singles2: 1870. Coincidence: 0.00.
dur: 14. pix: 0, 9. Singles1: 372. Singles2: 2289. Coincidence: 0.00.
dur: 15. pix: 0, 10. Singles1: 348. Singles2: 2380. Coincidence: 0.00.
dur: 17. pix: 0, 11. Singles1: 355. Singles2: 2457. Coincidence: 0.00.
dur: 19. pix: 0, 12. Singles1: 365. Singles2: 2230. Coincidence: 0.00.
dur: 20. pix: 0, 13. Singles1: 353. Singles2: 1963. Coi

# Long two-photon scan

In [6]:
masks_path = glob.glob(rf'{dir_path}\*diffuser_d{d_no}d*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
mplc.load_masks(masks, linear_tilts=True)

# Put idler in OK place 
mxi.move_absolute(8.95)
myi.move_absolute(2.1)

# scan signal
start_x = 11
end_x = 12.1
start_y = 8.7
end_y = 10.3
integration_time = 60
tt.set_integration_time(integration_time)

x_pixels = 22
y_pixels = 32
pixel_size_x = 0.05
pixel_size_y = 0.05


timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = f'{dir_path}\\{timestamp}_two_ph_speckles_d{d_no}d_exp_{integration_time}s.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, 
                        is_timetagger=True, coin_window=2e-9, saveto_path=path)

single1s, single2s, coincidences = scanner.scan(ph=tt , x_motor=mxs, y_motor=mys)


t = run_in_thread(scanner_sig.scan, x_motor=mxs, y_motor=mys, ph=tt)
t2 = run_in_thread(scanner_idl.scan, x_motor=mxi, y_motor=myi, ph=tt)    
t.join()
t2.join()

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 4531. Singles2: 764. Coincidence: 0.03.
dur: 60. pix: 0, 1. Singles1: 4527. Singles2: 802. Coincidence: 0.02.
dur: 121. pix: 0, 2. Singles1: 4513. Singles2: 868. Coincidence: 0.02.
dur: 181. pix: 0, 3. Singles1: 4506. Singles2: 958. Coincidence: 0.02.
dur: 242. pix: 0, 4. Singles1: 4529. Singles2: 1025. Coincidence: 0.03.
dur: 303. pix: 0, 5. Singles1: 4513. Singles2: 1042. Coincidence: 0.03.
dur: 363. pix: 0, 6. Singles1: 4522. Singles2: 1070. Coincidence: 0.03.
dur: 424. pix: 0, 7. Singles1: 4516. Singles2: 1102. Coincidence: 0.02.
dur: 484. pix: 0, 8. Singles1: 4522. Singles2: 1138. Coincidence: 0.02.
dur: 545. pix: 0, 9. Singles1: 4507. Singles2: 1168. Coincidence: 0.02.
dur: 605. pix: 0, 10. Singles1: 4510. Singles2: 1176. Coincidence: 0.07.
dur: 666. pix: 0, 11. Singles1: 4518. Singles2: 1140. Coincidence: 0.00.
dur: 726. pix: 0, 12. Singles1: 4515. Singles2: 1104. Coincidence: 0.03.
dur: 787. pix: 0, 13. S

In [34]:
from pianoq.misc.mplt import mimshow
from pianoq_results.scan_result import ScanResult
path = glob.glob(rf'{dir_path}\*speckles_d{d_no}d*.scan')[1]
print(path)
r = ScanResult()
r.loadfrom(path)
r.show_singles()
# mimshow(r.single2s)
# mimshow(r.single1s)
# mimshow(r.coincidences)

# print(path)

# mimshow(scanner.result.single1s)
# mimshow(scanner.result.single2s)
# scanner.result.coincidences.shape

G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_14_26_42_signal_speckles_d1d.scan
